# Advanced Analytics and Applications
## Analytics Anacondas 

### Data Collection and Preparation 

##### Part 02

In [1]:
# import relevant libaries 
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
import glob 
from vincenty import vincenty
import datetime
import warnings
warnings.filterwarnings(action='ignore')

#### Which weather station is suitable as the data source for different weather features?

<i> Deutscher Wetterdienst </i> has several weather stations in Germany, which can be looked up [here](https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/hourly/air_temperature/historical/TU_Stundenwerte_Beschreibung_Stationen.txt)

There are 5 weather stations in and around the city of Bonn. Unfortunately the stations in Bonn don't supply data for the year 2019. 

<img src='Data/bonn.jpg'/>

We have to check which of the remaining stations are closer to the city center of Bonn. For this we can use the functions of the libary <i> vincenty <i/>.

In [2]:
# distance in kilometers from Köln-Bonn to Bonn city center 
vincenty([50.8646,7.1575],[50.7285,7.0839])

16.004915

In [3]:
# distance in kilometers from Königswinter-Heiderhof to Bonn city center 
vincenty([50.7290,7.2047],[50.7285,7.0839])

8.529407

#### Reading the weather data for Königswinter-Heiderhof

In [4]:
# read in temperature data for 2019, select useful columns
temp = pd.read_csv("Data/Temperature.txt", sep = ";")
temp.rename(columns = {"TT_TU":"temperature °C", "MESS_DATUM":"datetime", "RF_TU": "relative_humidity"}, inplace=True)
temp.drop(labels=["STATIONS_ID", "QN_9", "eor"], axis=1, inplace=True)
temp = temp[(temp["datetime"] >= 2019010100) & (temp["datetime"] <= 2019123123)]
temp.reset_index(drop=True, inplace=True)
temp

,datetime,temperature °C,relative_humidity
0,2019010100,7.6,94.0
1,2019010101,7.5,93.0
2,2019010102,7.4,92.0
3,2019010103,7.5,85.0
4,2019010104,6.8,94.0
...,...,...,...
8755,2019123119,0.8,99.0
8756,2019123120,1.3,99.0
8757,2019123121,0.7,99.0
8758,2019123122,0.6,100.0


In [5]:
# read in precipitation data for 2019, select useful columns
# Köln Bonn used, because the data for precipitation in Königswinter is not completely for 2019
precipitation = pd.read_csv("Data/Precipitation.txt", sep = ";")
precipitation.rename(columns = {"  R1":"precipitaion_level (mm)", "MESS_DATUM":"datetime", "RS_IND": "precipitation y/n"}, inplace=True)
precipitation.drop(labels=["STATIONS_ID", "QN_8", "eor","WRTR"], axis=1, inplace=True)
precipitation = precipitation[(precipitation["datetime"] >= 2019010100) & (precipitation["datetime"] <= 2019123123)]
precipitation.reset_index(drop=True, inplace=True)
precipitation

,datetime,precipitaion_level (mm),precipitation y/n
0,2019010100,0.0,0
1,2019010101,0.0,0
2,2019010102,0.0,0
3,2019010103,0.0,1
4,2019010104,0.0,1
...,...,...,...
8755,2019123119,0.0,0
8756,2019123120,0.0,0
8757,2019123121,0.0,0
8758,2019123122,0.0,0


In [6]:
# read in wind data for 2019, select useful columns
# data for 36 hours missing (result = 8723 rows instead of 8760)
wind = pd.read_csv("Data/Wind.txt", sep = ";")
wind.rename(columns = {"   F":"average_wind_speed (m/s)", "MESS_DATUM":"datetime"}, inplace=True)
wind.drop(labels=["STATIONS_ID", "QN_3", "eor","   D"], axis=1, inplace=True)
wind = wind[(wind["datetime"] >= 2019010100) & (wind["datetime"] <= 2019123123)]
wind.reset_index(drop=True, inplace=True)
wind

,datetime,average_wind_speed (m/s)
0,2019010100,1.8
1,2019010101,2.0
2,2019010102,2.4
3,2019010103,3.3
4,2019010104,3.3
...,...,...
8718,2019123119,1.1
8719,2019123120,1.5
8720,2019123121,1.5
8721,2019123122,1.3


In [7]:
# concatenate and merge weather data to final dateframe 
weather = pd.concat([temp, precipitation.drop("datetime",axis=1)],axis=1)
weather = pd.merge(weather, wind,on="datetime",how="left")

# wind data for 36 hours missing (result = 8723 rows instead of 8760)
weather.isna().any(axis=0)

datetime                    False
temperature °C              False
relative_humidity           False
precipitaion_level (mm)     False
precipitation y/n           False
average_wind_speed (m/s)     True
dtype: bool

In [8]:
weather.to_csv("Data/final_weather.csv")
weather

,datetime,temperature °C,relative_humidity,precipitaion_level (mm),precipitation y/n,average_wind_speed (m/s)
0,2019010100,7.6,94.0,0.0,0,1.8
1,2019010101,7.5,93.0,0.0,0,2.0
2,2019010102,7.4,92.0,0.0,0,2.4
3,2019010103,7.5,85.0,0.0,1,3.3
4,2019010104,6.8,94.0,0.0,1,3.3
...,...,...,...,...,...,...
8755,2019123119,0.8,99.0,0.0,0,1.1
8756,2019123120,1.3,99.0,0.0,0,1.5
8757,2019123121,0.7,99.0,0.0,0,1.5
8758,2019123122,0.6,100.0,0.0,0,1.3


In [9]:
trip_data = pd.read_csv('Data/final_trips.csv',index_col=0)


In [10]:
def formatDatetimeForMerging(x):
    # return as integer for merging 
    return int(x[:13].replace('-','').replace(' ',''))

In [11]:
trip_data["datetime_start_for_merge_with_data"] = trip_data["datetime_start"].apply(lambda x: formatDatetimeForMerging(x))
trip_data

,index_start,p_place_type_start,datetime_start,b_number_start,trip_start,p_uid_start,p_bikes_x,latitude_start,longitude_start,p_name_start,...,p_number_end,p_bike_end,trip_duration,weekday,distance,weekend,day,month,hour,datetime_start_for_merge_with_data
0,3,12,2019-01-20 03:20:00,44952,start,12040694,1,50.703284,7.154586,BIKE 44952,...,0.0,True,17,6,1.543753,True,20,1,3,2019012003
1,5,12,2019-01-20 11:51:00,44952,start,12102064,1,50.716753,7.160890,BIKE 44952,...,4890.0,False,35,6,4.074607,True,20,1,11,2019012011
2,7,12,2019-01-20 14:32:00,44952,start,12112947,1,50.737916,7.112345,BIKE 44952,...,4743.0,False,29,6,4.252452,True,20,1,14,2019012014
3,15,12,2019-01-20 02:21:00,44715,start,12099152,1,50.752700,7.089911,BIKE 44715,...,0.0,True,12,6,1.465720,True,20,1,2,2019012002
4,17,12,2019-01-20 10:50:00,44715,start,12101268,1,50.740355,7.097171,BIKE 44715,...,0.0,True,9,6,1.056341,True,20,1,10,2019012010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1049367,2596229,12,2019-12-31 20:39:00,45362,start,26515236,1,50.722604,7.087977,BIKE 45362,...,0.0,True,4,1,0.471054,False,31,12,20,2019123120
1049368,2596231,12,2019-12-31 22:25:00,45362,start,26516964,1,50.723062,7.081308,BIKE 45362,...,0.0,True,14,1,1.697346,False,31,12,22,2019123122
1049369,2596243,12,2019-12-31 19:11:00,45291,start,26514879,1,50.720013,7.096394,BIKE 45291,...,0.0,True,18,1,2.938191,False,31,12,19,2019123119
1049370,2596251,0,2019-12-31 17:33:00,45200,start,10044315,6,50.717880,7.125460,Bundeshaus,...,0.0,True,27,1,3.184349,False,31,12,17,2019123117


In [12]:
# final dataframe, trips with weather data 
trips_weather = pd.merge(trip_data, weather, left_on="datetime_start_for_merge_with_data", right_on="datetime")

In [14]:
trips_weather.head(5)

,index_start,p_place_type_start,datetime_start,b_number_start,trip_start,p_uid_start,p_bikes_x,latitude_start,longitude_start,p_name_start,...,day,month,hour,datetime_start_for_merge_with_data,datetime,temperature °C,relative_humidity,precipitaion_level (mm),precipitation y/n,average_wind_speed (m/s)
0,3,12,2019-01-20 03:20:00,44952,start,12040694,1,50.703284,7.154586,BIKE 44952,...,20,1,3,2019012003,2019012003,-5.6,86.0,0.0,0,2.4
1,25,12,2019-01-20 03:08:00,44931,start,12093216,1,50.747953,7.119311,BIKE 44931,...,20,1,3,2019012003,2019012003,-5.6,86.0,0.0,0,2.4
2,75,0,2019-01-20 03:06:00,44594,start,10044445,2,50.736217,7.094604,Alter Friedhof,...,20,1,3,2019012003,2019012003,-5.6,86.0,0.0,0,2.4
3,177,12,2019-01-20 03:52:00,44966,start,12069877,1,50.708533,7.105807,BIKE 44966,...,20,1,3,2019012003,2019012003,-5.6,86.0,0.0,0,2.4
4,409,12,2019-01-20 03:30:00,44637,start,12098186,1,50.732503,7.097200,BIKE 44637,...,20,1,3,2019012003,2019012003,-5.6,86.0,0.0,0,2.4


In [15]:
trips_weather.to_csv("Data/final_trips_and_weather.csv")